In [1]:
!pip install datasets
!pip install optimum
!pip install onnxruntime
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from huggingface_hub import login
login("token")

In [3]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm

from datasets import load_dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, pipeline

from sklearn.metrics import accuracy_score, f1_score

In [5]:
import os
import time

import numpy as np
import torch

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
    AutoConfig
)
from optimum.onnxruntime import ORTModelForSequenceClassification, ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from collections import Counter

In [6]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
clinc = load_dataset("DeepPavlov/clinc_oos", "plus")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/74.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/133k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [7]:
sample = clinc["train"][0]
print(sample)

# Каждый объект датасета содержит текст и соответствующее ему намерение.

{'text': 'what expression would i use to say i love you if i were an italian', 'label': 61, 'label_text': 'translate'}


In [8]:
print(clinc)
print(clinc["train"].features)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 15250
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 3100
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 5500
    })
})
{'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'label_text': Value(dtype='string', id=None)}


In [9]:
# Number of unique labels
labels = clinc["train"].unique("label")
num_labels = len(labels)
print(f"Number of labels: {num_labels}")

Number of labels: 151


In [10]:
# Получаем уникальные "label_text" и создаем id2label
unique_intents = sorted(list(set(clinc["train"]["label_text"])))
id2label = {i: intent for i, intent in enumerate(unique_intents)}
label2id = {intent: i for i, intent in enumerate(unique_intents)}

In [11]:
num_labels = len(id2label)
print(f"Количество классов: {num_labels}")
print("id2label:", id2label)

Количество классов: 151
id2label: {0: 'accept_reservations', 1: 'account_blocked', 2: 'alarm', 3: 'application_status', 4: 'apr', 5: 'are_you_a_bot', 6: 'balance', 7: 'bill_balance', 8: 'bill_due', 9: 'book_flight', 10: 'book_hotel', 11: 'calculator', 12: 'calendar', 13: 'calendar_update', 14: 'calories', 15: 'cancel', 16: 'cancel_reservation', 17: 'car_rental', 18: 'card_declined', 19: 'carry_on', 20: 'change_accent', 21: 'change_ai_name', 22: 'change_language', 23: 'change_speed', 24: 'change_user_name', 25: 'change_volume', 26: 'confirm_reservation', 27: 'cook_time', 28: 'credit_limit', 29: 'credit_limit_change', 30: 'credit_score', 31: 'current_location', 32: 'damaged_card', 33: 'date', 34: 'definition', 35: 'direct_deposit', 36: 'directions', 37: 'distance', 38: 'do_you_have_pets', 39: 'exchange_rate', 40: 'expiration_date', 41: 'find_phone', 42: 'flight_status', 43: 'flip_coin', 44: 'food_last', 45: 'freeze_account', 46: 'fun_fact', 47: 'gas', 48: 'gas_type', 49: 'goodbye', 50: '

In [12]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
# Define checkpoint
checkpoint = "ASCCCCCCCC/distilbert-base-uncased-finetuned-clinc"

In [14]:
#Токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)

#Загружаем модель и обновляем config.id2label
config = AutoConfig.from_pretrained(checkpoint)
id2label = config.id2label
label2id = config.label2id
num_labels = len(id2label)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config).to(device)

print("Модель загружена. Количество параметров:", model.num_parameters())

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/8.20k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

Модель загружена. Количество параметров: 67069591


In [15]:
# Create pipeline
pipe = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=0,
    truncation=True,
    max_length=128,
    function_to_apply="softmax"
)

# Sample input
sample_input = clinc['test']['text'][101]
print("Текст примера:", sample_input)
print("Правильный intent:", clinc['test']['label_text'][101])

Device set to use cpu


Текст примера: what does amicable mean
Правильный intent: definition


In [16]:
# Warmup
for _ in range(10):
    pipe(sample_input)

# Измеряем скорость работы модели
start = time.time()
for _ in range(100):
    pipe(sample_input)
total_time_model = time.time() - start
print("Время обработки 100 запросов (PyTorch):", total_time_model)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Время обработки 100 запросов (PyTorch): 3.2381341457366943


In [17]:
# Подготовка тестовой выборки
data_test_X = clinc['test']['text'][::50]
data_test_y = clinc['test']['label_text'][::50]  # ИСПРАВЛЕНО

from collections import Counter
# Функция оценки точности
def show_accuracy(pipe_func):
    preds = []
    truths = []

    for i, text in enumerate(tqdm(data_test_X)):
        result = pipe_func(text)[0]['label']
        if result.startswith("LABEL_"):
            idx = int(result.split("_")[1])
            pred_intent = id2label.get(idx, "Unknown")
        else:
            pred_intent = result
        true_intent = data_test_y[i]

        preds.append(pred_intent.lower())
        truths.append(true_intent.lower())

        # Выводим первые 5 для проверки
        if i < 5:
            print(f"Input: {text[:50]}...")
            print(f"Predicted: {pred_intent} | True: {true_intent}")

    accuracy = accuracy_score(truths, preds)
    print(f"\nAccuracy: {accuracy:.4f}")
    print("\nTop predicted intents:", Counter(preds).most_common(5))
    print("Top true intents:     ", Counter(truths).most_common(5))

# Оценка базовой модели
print("Оценка точности базовой модели...")
show_accuracy(pipe)
from collections import Counter


Оценка точности базовой модели...


  0%|          | 0/110 [00:00<?, ?it/s]

Input: how would you say fly in italian...
Predicted: spelling | True: translate
Input: how much money can i transfer today...
Predicted: exchange_rate | True: transfer
Input: what does assiduous mean...
Predicted: oos | True: definition
Input: i would like to change my insurance policy...
Predicted: insurance_change | True: insurance_change
Input: find my cellphone for me!...
Predicted: find_phone | True: find_phone

Accuracy: 0.5818

Top predicted intents: [('oos', 23), ('oil_change_how', 4), ('international_visa', 4), ('exchange_rate', 3), ('calories', 3)]
Top true intents:      [('oos', 20), ('translate', 1), ('transfer', 1), ('definition', 1), ('insurance_change', 1)]


In [18]:
print(pipe(sample_input))

[{'label': 'thank_you', 'score': 0.01456065010279417}]


In [19]:
# Сохраняем оптимизированную модель
save_dir = "optimized_distilbert_clinc"

# Экспорт в ONNX и оптимизация
ort_model = ORTModelForSequenceClassification.from_pretrained(
    checkpoint,
    export=True,
    provider="CPUExecutionProvider",
    config=config
)

optimizer = ORTOptimizer.from_pretrained(ort_model)
optimization_config = OptimizationConfig(optimization_level=99)
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)

# Загружаем оптимизированную модель
optimized_model = ORTModelForSequenceClassification.from_pretrained(save_dir)
optimized_model.config.id2label = id2label
optimized_model.config.label2id = label2id

# Создаем пайплайн с явным указанием CPUExecutionProvider
onnx_pipe = pipeline(
    "text-classification",
    model=optimized_model,
    tokenizer=tokenizer
)

# Проверка
print(onnx_pipe("I want to check my flight status"))

/usr/local/lib/python3.11/dist-packages/optimum/onnxruntime/configuration.py:784: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
Device set to use cpu


[{'label': 'book_flight', 'score': 0.018362144008278847}]


In [20]:
# Save optimized model directory
save_dir = "optimized_bert_clinc"

In [21]:
print(optimized_model.providers)

['CPUExecutionProvider']


In [23]:
# Export and optimize model
print("Exporting and optimizing model with Optimum...")
ort_model = ORTModelForSequenceClassification.from_pretrained(
    checkpoint,
    export=True,
    provider="CPUExecutionProvider",
    config=config
)

optimizer = ORTOptimizer.from_pretrained(ort_model)
optimization_config = OptimizationConfig(optimization_level=99)
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)

Exporting and optimizing model with Optimum...


PosixPath('optimized_bert_clinc')

In [26]:
# Load optimized model
optimized_model = ORTModelForSequenceClassification.from_pretrained(save_dir)
optimized_model.config.id2label = id2label
optimized_model.config.label2id = label2id

onnx_pipe = pipeline(
    "text-classification",
    model=optimized_model,
    tokenizer=tokenizer,
)

Device set to use cpu


In [27]:
# Warmup optimized model
for _ in range(10):
    onnx_pipe(sample_input)

# Benchmark optimized model
start = time.time()
for _ in range(100):
    onnx_pipe(sample_input)
total_time_optimized = time.time() - start
print("Time for 100 runs (Optimized bert_clinc):", total_time_optimized)

# Evaluate accuracy of optimized model
print("Evaluating optimized model accuracy...")
show_accuracy(onnx_pipe)

Time for 100 runs (Optimized bert_clinc): 2.0423245429992676
Evaluating optimized model accuracy...


  0%|          | 0/110 [00:00<?, ?it/s]

Input: how would you say fly in italian...
Predicted: spelling | True: translate
Input: how much money can i transfer today...
Predicted: exchange_rate | True: transfer
Input: what does assiduous mean...
Predicted: oos | True: definition
Input: i would like to change my insurance policy...
Predicted: insurance_change | True: insurance_change
Input: find my cellphone for me!...
Predicted: find_phone | True: find_phone

Accuracy: 0.5818

Top predicted intents: [('oos', 23), ('oil_change_how', 4), ('international_visa', 4), ('exchange_rate', 3), ('calories', 3)]
Top true intents:      [('oos', 20), ('translate', 1), ('transfer', 1), ('definition', 1), ('insurance_change', 1)]
